# Machine learning Grid Search

## Imports

In [68]:
import sys
import cufflinks
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
import pickle

warnings.filterwarnings('ignore')

sys.path.append('./..')
cufflinks.go_offline()

In [69]:
from Corpus.Corpus import get_corpus, filter_binary_pn, filter_corpus_small
from auxiliar.VectorizerHelper import vectorizer, vectorizerIdf, preprocessor
from auxiliar import parameters
from auxiliar.HtmlParser import HtmlParser

In [70]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
import copy

## Config

In [71]:
polarity_dim = 5
clasificadores=['lr', 'ls', 'mb', 'rf']
idf = True
target_names=['Neg', 'Pos']
kfolds = 10
base_dir = '2-clases' if polarity_dim == 2 else ('3-clases' if polarity_dim == 3 else '5-clases')
name = 'machine_learning/tweeter/grid_search'

## Get data

In [72]:
cine = HtmlParser(200, "http://www.muchocine.net/criticas_ultimas.php", 1)
data_corpus = get_corpus('general-corpus', 'general-corpus', 1, None)

if polarity_dim == 2:
    data_corpus = filter_binary_pn(data_corpus)
    cine = filter_binary_pn(cine.get_corpus())
elif polarity_dim == 3:
    data_corpus = filter_corpus_small(data_corpus)
    cine = filter_corpus_small(cine.get_corpus())
elif polarity_dim == 5:
    cine = cine.get_corpus()
cine = cine[:5000]
used_data = pd.DataFrame(data_corpus)

#Intentando obtener datos del archivo csv...
./../Corpus/../data/general-corpus.csv
#Datos recuperados!


In [79]:
cine['pol_name'] = cine[['polarity']].applymap(lambda x: 'Neg+' if x == 1 else 'Neg' if x == 2 else 'Neu' if x == 3 else 'Pos' if x == 4 else 'Pos+')

In [88]:
cine.groupby('pol_name').sum().reset_index().reindex([1,0,2,3,4,5]).set_index('pol_name').iplot(kind='bar')

In [67]:
used_data[used_data.polarity == 2].iloc[77].content

'Habrá que ir calentando los motores para esta noche... Y no, no es que vaya a ver el clásico ;)'

## Split data

In [9]:
split = used_data.shape[0] * 0.7
train_corpus = used_data.loc[:split - 1 , :]
test_corpus = used_data.loc[split:, :]

## Initialize ML

In [10]:
vect = vectorizerIdf if idf else vectorizer
ls = CalibratedClassifierCV(LinearSVC()) if polarity_dim == 2 else OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()))
lr = LogisticRegression(solver='lbfgs') if polarity_dim == 2 else OneVsRestClassifier(LogisticRegression())
mb = MultinomialNB() if polarity_dim == 2 else OneVsRestClassifier(MultinomialNB())
rf = RandomForestClassifier() if polarity_dim == 2 else OneVsRestClassifier(RandomForestClassifier())

In [11]:
pipeline_ls = Pipeline([
    ('prep', copy.deepcopy(preprocessor)),
    ('vect', copy.deepcopy(vect)),
    ('ls', ls)
])
pipeline_lr = Pipeline([
    ('prep', copy.deepcopy(preprocessor)),
    ('vect', copy.deepcopy(vect)),
    ('lr', lr)
])
pipeline_mb = Pipeline([
    ('prep', copy.deepcopy(preprocessor)),
    ('vect', copy.deepcopy(vect)),
    ('mb', mb)
])
pipeline_rf = Pipeline([
    ('prep', copy.deepcopy(preprocessor)),
    ('vect', copy.deepcopy(vect)),
    ('rf', rf)
])

In [12]:
pipelines = {
    'ls': pipeline_ls,
    'lr': pipeline_lr,
    'mb': pipeline_mb,
    'rf': pipeline_rf
}
pipelines_train = {
    'ls': ls,
    'lr': lr,
    'mb': mb,
    'rf': rf
}

In [13]:
params = parameters.parameters_bin if polarity_dim == 2 else parameters.parameters

In [14]:
params

{'ls': {'ls__estimator__base_estimator__max_iter': [1000, 1200, 1500]},
 'lr': {'lr__estimator__max_iter': [100, 200, 500]},
 'mb': {'mb__estimator__alpha': [1.0, 0.5, 1.5]},
 'rf': {'rf__estimator__n_estimators': [10, 100]}}

## Train

In [12]:
folds = pd.read_pickle('./../data/pkls/folds.pkl') # k-folds precargados
folds = folds.values

In [13]:
pd.Series(train_corpus.content.values)

0                               @marodriguezb Gracias MAR
1       Off pensando en el regalito Sinde, la que se v...
2       Conozco a alguien q es adicto al drama! Ja ja ...
3       Toca @crackoviadeTV3 . Grabación dl especial N...
4       Buen día todos! Lo primero mandar un abrazo gr...
5       Desde el escaño. Todo listo para empezar #endi...
6       Bdías. EM no se ira de puente. Si vosotros os ...
7       Un sistema económico q recorta dinero para pre...
8                       #programascambiados caca d ajuste
9                                            Buen viernes
10      “@adri_22_22: #programascambiados es TT gracia...
11      La Universidad confía en De la Calle para enca...
12      ¿Me ayudáis a que #indultoneiro sea TT? Por si...
13      abcdesevilla.es: Recio no tiene «indicios pote...
14      abcdesevilla.es: Cuatro altos cargos de Empleo...
15      La marcha atrás del PP en posponer devolución ...
16      Viernes negro: Aumenta el paro en Noviembre y ...
17      Accide

In [14]:
results = {}
grids = {}
with tqdm(total=len(clasificadores) * 10) as pbar:
    for c in clasificadores:
        results[c] = { 'real': {}, 'predicted': {} }
        i = 0
        params[c].update(parameters.vect_params)
        params[c].update(parameters.prepro_params)
        param_grid = params[c]
        grid_search = GridSearchCV(pipelines[c], param_grid, verbose=2, scoring='accuracy', refit=False, cv=3)
        grid = grid_search.fit(train_corpus.content, train_corpus.polarity)
        grids[c] = grid
        best_parameters = grid.best_params_
        train_params = {}
        for param_name in sorted(parameters.vect_params.keys()):
            train_params.update({param_name[6:]: best_parameters[param_name]})
        vect.set_params(**train_params)
        preprocessor.set_params(**train_params)
        x_prepro = preprocessor.fit_transform(train_corpus.content)
        x_vect = vect.fit_transform(x_prepro, train_corpus.polarity).toarray()
        for train_index, test_index in folds:
            train_x = x_vect[train_index]
            train_y = train_corpus.polarity[train_index]
            test_x = x_vect[test_index]
            test_y = train_corpus.polarity[test_index]

            pipelines_train[c].fit(train_x, train_y)

            predicted = pipelines_train[c].predict(test_x)
            
            results[c]['real'][i] = test_y.values.tolist()
            results[c]['predicted'][i] = predicted.tolist()
            i = i + 1

            pbar.update(1)

    

  0%|          | 0/40 [00:00<?, ?it/s][Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0> 
[CV]  lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0>, total=  13.9s
[CV] lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0> 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.0s remaining:    0.0s


[CV]  lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0>, total=  13.6s
[CV] lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0> 
[CV]  lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0>, total=  13.8s
[CV] lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f847d982158> 
[CV]  lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f847d982158>, total=   8.9s
[CV] lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f847d982158> 
[CV]  lr__estimator__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f847d982158>, total

[CV]  lr__estimator__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0>, total=  16.0s
[CV] lr__estimator__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0> 
[CV]  lr__estimator__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0>, total=  15.3s
[CV] lr__estimator__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0> 
[CV]  lr__estimator__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f847d9820d0>, total=  17.6s
[CV] lr__estimator__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f847d982158> 
[CV]  lr__estimator__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f847d982158>, total= 

...

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  7.0min finished
 75%|███████▌  | 30/40 [27:18<01:18,  7.81s/it] [Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function stemize at 0x7f847d9820d0> 
[CV]  prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function stemize at 0x7f847d9820d0>, total=  16.6s
[CV] prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function stemize at 0x7f847d9820d0> 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.6s remaining:    0.0s


[CV]  prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function stemize at 0x7f847d9820d0>, total=  16.8s
[CV] prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function stemize at 0x7f847d9820d0> 
[CV]  prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function stemize at 0x7f847d9820d0>, total=  17.8s
[CV] prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function tokenize at 0x7f847d982158> 
[CV]  prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function tokenize at 0x7f847d982158>, total=  13.3s
[CV] prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function tokenize at 0x7f847d982158> 
[CV]  prep__negation=0, prep__repeated_letters=True, rf__estimator__n_estimators=10, vect__tokenizer=<function tokenize at 0

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 12.5min finished
100%|██████████| 40/40 [59:32<00:00, 109.85s/it]


In [15]:
results

{'lr': {'real': {0: [4,
    5,
    4,
    5,
    5,
    2,
    5,
    4,
    5,
    2,
    5,
    5,
    4,
    2,
    2,
    4,
    4,
    4,
    1,
    5,
    2,
    2,
    5,
    5,
    5,
    5,
    1,
    2,
    4,
    4,
    4,
    5,
    2,
    5,
    4,
    2,
    2,
    2,
    5,
    3,
    5,
    1,
    2,
    4,
    5,
    5,
    2,
    5,
    2,
    4,
    1,
    2,
    2,
    5,
    5,
    2,
    4,
    5,
    1,
    4,
    5,
    4,
    4,
    2,
    2,
    4,
    2,
    2,
    1,
    2,
    5,
    5,
    1,
    4,
    2,
    5,
    3,
    5,
    5,
    5,
    4,
    5,
    1,
    2,
    5,
    5,
    2,
    4,
    5,
    3,
    4,
    4,
    1,
    1,
    5,
    2,
    5,
    3,
    2,
    2,
    5,
    5,
    2,
    1,
    2,
    1,
    1,
    2,
    5,
    5,
    4,
    1,
    4,
    1,
    1,
    5,
    4,
    5,
    4,
    2,
    5,
    2,
    5,
    5,
    5,
    4,
    1,
    5,
    1,
    2,
    1,
    1,
    1,
    4,
    5,
    5,
    1,
    5,
    4,
    2,
   

In [16]:
pd.DataFrame(results).to_pickle('../results/'+name+'/'+base_dir+'/results-idf.pkl')

In [17]:
with open('../results/'+name+'/'+base_dir+'/grid_results-idf.pkl', 'wb') as fp:
    pickle.dump(grids, fp)

In [15]:
with open('../results/'+name+'/'+base_dir+'/grid_results-idf.pkl', 'rb') as fp:
    grids = pickle.load(fp)

In [16]:
test_results = {}
with tqdm(total=len(clasificadores)) as pbar:
    for c in clasificadores:
        test_results[c] = { 'real': {}, 'cine_real': {}, 'predicted': {}, 'cine_predicted': {} }
        i = 0
        grid = grids[c]
        best_parameters = grid.best_params_
        train_params = {}
        for param_name in sorted(parameters.vect_params.keys()):
            train_params.update({param_name[6:]: best_parameters[param_name]})
        vect.set_params(**train_params)
        vect.fit(data_corpus.content, data_corpus.polarity)
        x_vect = vect.transform(train_corpus.content).toarray()
        x_vect_test = vect.transform(test_corpus.content).toarray()
        x_vect_cine = vect.transform(cine.content).toarray()
        train_x = x_vect
        train_y = train_corpus.polarity
        test_x = x_vect_test
        test_y = test_corpus.polarity
        cine_y = cine.polarity

        pipelines_train[c].fit(train_x, train_y)

        predicted = pipelines_train[c].predict(test_x)
        cine_predicted = pipelines_train[c].predict(x_vect_cine)

        test_results[c]['real'][i] = test_y.values.tolist()
        test_results[c]['cine_real'][i] = cine_y.values.tolist()
        test_results[c]['predicted'][i] = predicted.tolist()
        test_results[c]['cine_predicted'][i] = cine_predicted.tolist()
        i = i + 1

        pbar.update(1)
        pbar.update(1)

8it [28:39, 214.91s/it]                       


In [17]:
pd.DataFrame(test_results).to_pickle('../results/'+name+'/'+base_dir+'/test_results-idf.pkl')